In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import akey

In [19]:
file = 'api-data/stock_prices.csv'
stock_prices = pd.read_csv(file)
company_file = 'api-data/company_info.csv'
company = pd.read_csv(company_file)
#Setting date column to be date/time object for time series analysis
stock_prices['index.1'] = pd.to_datetime(stock_prices['index.1'], format='%Y-%m-%d') 
#Renaming columns 
stock_prices.rename(columns={'index':'old_index', 'index.1': 'Date', '1. open': 'Open', '2. high':'High', 
                             '3. low':'Low','4. close':'Close', '5. volume':'Daily Volume'}, inplace=True)
#dropping un-needed columns in existing dataframe
stock_prices.drop(columns=['old_index'], inplace=True)
#setting index to Date for analysis
stock_prices.set_index('Date', inplace=True)
stock_prices['Year'] = pd.DatetimeIndex(stock_prices.index).year
stock_prices['Month'] = pd.DatetimeIndex(stock_prices.index).month
stock_prices.to_csv('api-data/cleaned_stock_prices.csv')
stock_prices.reset_index(inplace=True)
stock_prices.head()

,Date,Open,High,Low,Close,Daily Volume,Ticker,Year,Month
0,2020-12-15,21.70,22.11,16.580,17.99,23931715.0,AAN,2020,12
1,2020-12-15,21.70,22.11,16.580,17.99,23931715.0,AAN,2020,12
2,2020-12-15,8.33,8.53,7.620,8.41,6979264.0,AAOI,2020,12
3,2020-11-30,8.82,10.20,7.200,8.31,17670623.0,AAOI,2020,11
4,2020-10-30,11.40,11.94,8.469,8.78,8911887.0,AAOI,2020,10


In [9]:
company.head()
#stock_prices.head()

,Ticker,Industry,Sector,Full Time Emp
0,AAOI,Technology,Semiconductors,3115
1,AAON,Industrials,Building Products & Equipment,2290
2,AAT,Real Estate,REIT-Diversified,199
3,AAWW,Industrials,Airports & Air Services,3587
4,ABCB,Financial Services,Banks-Regional,2656


In [21]:
#adding company information to stock prices file
all_info= stock_prices.merge(company, on='Ticker', how='left')
all_info.set_index('Date', inplace=True)
all_info.sort_index(ascending=True, inplace=True)
all_info['Year'] = pd.DatetimeIndex(all_info.index).year
all_info['Month'] = pd.DatetimeIndex(all_info.index).month
#drop remaining unneeded columns
all_info.drop(columns=['Open','High','Low','Daily Volume'], inplace=True)
all_info.to_csv('api-data/all_info.csv')
all_info.head()

,Close,Ticker,Year,Month,Industry,Sector,Full Time Emp
Date,,,,,,,
1999-12-31,20.00,CLB,1999,12,Energy,Oil & Gas Equipment & Services,3800.0
1999-12-31,10.38,CENT,1999,12,Consumer Defensive,Packaged Foods,5600.0
1999-12-31,0.08,RDNT,1999,12,Healthcare,Diagnostics & Research,6165.0
1999-12-31,11.63,ESE,1999,12,Technology,Scientific & Technical Instruments,2713.0
1999-12-31,20.75,FBP,1999,12,Financial Services,Banks-Regional,2674.0


In [16]:
#This block loops through the date indices and creates separate files for each year for all stock 
#closing prices and outputs to CSV
years = ['2015','2016','2017','2018','2019','2020']
for year in years:
    year = all_info[f'{year}-01-01':f'{year}-12-31'].to_csv(f'api-data/yearly_summary/prices_for_{year}.csv')

In [27]:
#analyzing 2020 data to find interesting sectors
file='api-data/yearly_summary/prices_for_2020.csv'
stocks2020 = pd.read_csv(file)
#lamba function takes each stock ticker in the CSV, finds the first date and the last date and then calculates % chg
sector_perf = stocks2020.groupby(['Ticker','Industry','Sector'],sort=False).apply(lambda x: (x['Close'].values[-1] - x['Close'].values[0]) / x['Close'].values[-1] * 100)\
    .reset_index(name='pct change')
sector_perf.head()

,Ticker,Industry,Sector,pct change
0,TRST,Financial Services,Banks-Regional,-21.068702
1,LNTH,Healthcare,Diagnostics & Research,-22.791024
2,EPRT,Real Estate,REIT-Diversified,-30.174446
3,MDC,Consumer Cyclical,Residential Construction,14.384397
4,VRA,Consumer Cyclical,Footwear & Accessories,-22.978177


In [36]:
sectors = sector_perf.groupby('Sector').mean()
sectors.sort_values('pct change', ascending=False, inplace=True)

In [38]:
sectors.head(25)

,pct change
Sector,
Medical Distribution,78.195751
Internet Retail,66.645768
Airports & Air Services,60.231317
Home Improvement Retail,50.916958
Packaging & Containers,47.738883
Real Estate-Diversified,45.696562
Internet Content & Information,44.097934
Leisure,41.229907
Specialty Retail,37.964274


In [39]:
sectors.describe()

,pct change
count,120.000000
mean,2.784474
std,28.633243
min,-132.456140
25%,-11.060859
50%,6.322299
75%,18.634826
max,78.195751
